In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Esophageal_Cancer"
cohort = "GSE77790"

# Input paths
in_trait_dir = "../../input/GEO/Esophageal_Cancer"
in_cohort_dir = "../../input/GEO/Esophageal_Cancer/GSE77790"

# Output paths
out_data_file = "../../output/preprocess/Esophageal_Cancer/GSE77790.csv"
out_gene_data_file = "../../output/preprocess/Esophageal_Cancer/gene_data/GSE77790.csv"
out_clinical_data_file = "../../output/preprocess/Esophageal_Cancer/clinical_data/GSE77790.csv"
json_path = "../../output/preprocess/Esophageal_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Differentially expressed genes after miRNA or siRNA transfection in human cancer cell lines II"
!Series_summary	"To identify differentially expressed genes by anti cancer treatments (microRNAs or siRNAs) in human cancer, several cell lines (pancreatic cancer, esophageal cancer, bladder cancer, prostate cancer, renal cell carcinoma and lung squamous cell carcinoma) were subjected to Agilent whole genome microarrays."
!Series_overall_design	"Human cell lines (Panc-1, sw1990, TE8, TE9, A549, MRC-5, BOY, T24, PC3, C4-2, 786-O, A-498 and EBC-1) were treated with miRNAs (miR-375, miR-29a, miR-26a, miR-145-5p, miR-145-3p, miR-218, miR-320a), siRNAs (si-MMP11, si-LAMP1, si-LOXL2, si-PLOD2, si-UHRF1, and si-FOXM1)."
Sample Characteristics Dictionary:
{0: ['cell line: EBC-1', 'cell line: C4-2', 'cell line: PC3', 'cell line: A-498', 'cell line: 786-O', 'cell line: BOY', 'cell line: T24', 'cell line: A549', 'cell line: MRC-5', 'cell line: Panc-1', 'cell line:

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Analysis
# Based on the background information, this dataset appears to be gene expression data
# from microarray analysis, which is suitable for our study.
is_gene_available = True

# 2. Clinical Features Analysis
# 2.1 Data Availability
# For trait (esophageal cancer), we can use the cell type information (row 1)
# For age and gender, there's no information in the sample characteristics
trait_row = 1
age_row = None  # No age data available
gender_row = None  # No gender data available

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    """Convert cell type to binary trait (esophageal cancer or not)"""
    if not isinstance(value, str):
        return None
    
    # Extract the value after the colon
    if ':' in value:
        value = value.split(':', 1)[1].strip().lower()
    
    # Check if it's esophageal cancer
    if 'esophageal cancer' in value:
        return 1
    else:
        return 0

def convert_age(value):
    # Not used, as age data is not available
    return None

def convert_gender(value):
    # Not used, as gender data is not available
    return None

# 3. Save Metadata
# Check if trait data is available (trait_row is not None)
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available, 
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Only execute if trait_row is not None
if trait_row is not None:
    # Create a DataFrame from the sample characteristics dictionary provided in the previous output
    sample_characteristics_dict = {
        0: ['cell line: EBC-1', 'cell line: C4-2', 'cell line: PC3', 'cell line: A-498', 
            'cell line: 786-O', 'cell line: BOY', 'cell line: T24', 'cell line: A549', 
            'cell line: MRC-5', 'cell line: Panc-1', 'cell line: sw1990', 'cell line: TE8', 
            'cell line: TE9'],
        1: ['cell type: lung squamous cell carcinoma', 'cell type: prostate cancer', 
            'cell type: bladder cancer', 'cell type: renal cell carcinoma', 
            'cell type: lung fibroblast', 'cell type: pancreatic cancer', 
            'cell type: esophageal cancer'],
        2: ['transfection: no transfection']
    }
    
    import pandas as pd
    # Creating the clinical_data DataFrame from the dictionary
    # We need to transpose the data to get samples as rows and features as columns
    clinical_data = pd.DataFrame({col: values for col, values in enumerate(list(zip(*[values for values in sample_characteristics_dict.values()])))})
    
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the data
    preview = preview_df(selected_clinical_df)
    print("Clinical Data Preview:")
    print(preview)
    
    # Save clinical features to CSV
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Clinical Data Preview:
{0: [0.0]}
Clinical data saved to ../../output/preprocess/Esophageal_Cancer/clinical_data/GSE77790.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. First, let's examine the structure of the matrix file to understand its format
import gzip

# Peek at the first few lines of the file to understand its structure
with gzip.open(matrix_file, 'rt') as file:
    # Read first 100 lines to find the header structure
    for i, line in enumerate(file):
        if '!series_matrix_table_begin' in line:
            print(f"Found data marker at line {i}")
            # Read the next line which should be the header
            header_line = next(file)
            print(f"Header line: {header_line.strip()}")
            # And the first data line
            first_data_line = next(file)
            print(f"First data line: {first_data_line.strip()}")
            break
        if i > 100:  # Limit search to first 100 lines
            print("Matrix table marker not found in first 100 lines")
            break

# 3. Now try to get the genetic data with better error handling
try:
    gene_data = get_genetic_data(matrix_file)
    print(gene_data.index[:20])
except KeyError as e:
    print(f"KeyError: {e}")
    
    # Alternative approach: manually extract the data
    print("\nTrying alternative approach to read the gene data:")
    with gzip.open(matrix_file, 'rt') as file:
        # Find the start of the data
        for line in file:
            if '!series_matrix_table_begin' in line:
                break
                
        # Read the headers and data
        import pandas as pd
        df = pd.read_csv(file, sep='\t', index_col=0)
        print(f"Column names: {df.columns[:5]}")
        print(f"First 20 row IDs: {df.index[:20]}")
        gene_data = df


Found data marker at line 81
Header line: "ID_REF"	"GSM2059404"	"GSM2059405"	"GSM2059406"	"GSM2059407"	"GSM2059408"	"GSM2059409"	"GSM2059410"	"GSM2059411"	"GSM2059412"	"GSM2059413"	"GSM2059414"	"GSM2059415"	"GSM2059416"	"GSM2059417"	"GSM2059418"	"GSM2059419"	"GSM2059420"	"GSM2059421"	"GSM2059422"	"GSM2059423"	"GSM2059424"	"GSM2059425"	"GSM2059426"	"GSM2059427"	"GSM2059428"	"GSM2059429"	"GSM2059430"	"GSM2059431"	"GSM2059432"	"GSM2059433"	"GSM2059434"	"GSM2059435"
First data line: 1	-1.492678368e-001	9.385965497e-002	-8.941784384e-002	-1.349943700e-002	-1.599001264e-002	-8.062755446e-002	-5.685066626e-002	3.483449753e-002	1.110190735e-002	-1.109288193e-002	-3.863425129e-002	-4.031110222e-002	3.436493922e-002	6.242996551e-002	-3.869467488e-002	-2.818536224e-004	-6.648348866e-002	-7.110430995e-002	-1.601804138e-003	-6.578105194e-002	-9.610465045e-004	3.293553993e-002	5.540124407e-002	-7.305230142e-002	-1.253722506e-002	-6.620679603e-003	-7.651308691e-002	-5.726181154e-002	-2.069165415e-002

Index(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13',
       '14', '15', '16', '17', '18', '19', '20'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Looking at the gene identifiers in the gene expression data
# The identifiers are numerical (1, 2, 3, etc.) which are not standard human gene symbols
# These appear to be probe IDs that need to be mapped to gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Let's first examine the structure of the SOFT file before trying to parse it
import gzip

# Look at the first few lines of the SOFT file to understand its structure
print("Examining SOFT file structure:")
try:
    with gzip.open(soft_file, 'rt') as file:
        # Read first 20 lines to understand the file structure
        for i, line in enumerate(file):
            if i < 20:
                print(f"Line {i}: {line.strip()}")
            else:
                break
except Exception as e:
    print(f"Error reading SOFT file: {e}")

# 2. Now let's try a more robust approach to extract the gene annotation
# Instead of using the library function which failed, we'll implement a custom approach
try:
    # First, look for the platform section which contains gene annotation
    platform_data = []
    with gzip.open(soft_file, 'rt') as file:
        in_platform_section = False
        for line in file:
            if line.startswith('^PLATFORM'):
                in_platform_section = True
                continue
            if in_platform_section and line.startswith('!platform_table_begin'):
                # Next line should be the header
                header = next(file).strip()
                platform_data.append(header)
                # Read until the end of the platform table
                for table_line in file:
                    if table_line.startswith('!platform_table_end'):
                        break
                    platform_data.append(table_line.strip())
                break
    
    # If we found platform data, convert it to a DataFrame
    if platform_data:
        import pandas as pd
        import io
        platform_text = '\n'.join(platform_data)
        gene_annotation = pd.read_csv(io.StringIO(platform_text), delimiter='\t', 
                                      low_memory=False, on_bad_lines='skip')
        print("\nGene annotation preview:")
        print(preview_df(gene_annotation))
    else:
        print("Could not find platform table in SOFT file")
        
        # Try an alternative approach - extract mapping from other sections
        with gzip.open(soft_file, 'rt') as file:
            for line in file:
                if 'ANNOTATION information' in line or 'annotation information' in line:
                    print(f"Found annotation information: {line.strip()}")
                if line.startswith('!Platform_title') or line.startswith('!platform_title'):
                    print(f"Platform title: {line.strip()}")
            
except Exception as e:
    print(f"Error processing gene annotation: {e}")


Examining SOFT file structure:
Line 0: ^DATABASE = GeoMiame
Line 1: !Database_name = Gene Expression Omnibus (GEO)
Line 2: !Database_institute = NCBI NLM NIH
Line 3: !Database_web_link = http://www.ncbi.nlm.nih.gov/geo
Line 4: !Database_email = geo@ncbi.nlm.nih.gov
Line 5: ^SERIES = GSE77790
Line 6: !Series_title = Differentially expressed genes after miRNA or siRNA transfection in human cancer cell lines II
Line 7: !Series_geo_accession = GSE77790
Line 8: !Series_status = Public on Apr 13 2016
Line 9: !Series_submission_date = Feb 10 2016
Line 10: !Series_last_update_date = Oct 07 2019
Line 11: !Series_pubmed_id = 27633630
Line 12: !Series_pubmed_id = 27862697
Line 13: !Series_pubmed_id = 27072587
Line 14: !Series_pubmed_id = 27779648
Line 15: !Series_pubmed_id = 27765924
Line 16: !Series_pubmed_id = 29050264
Line 17: !Series_summary = To identify differentially expressed genes by anti cancer treatments (microRNAs or siRNAs) in human cancer, several cell lines (pancreatic cancer, esop


Gene annotation preview:
{'ID': [1, 2, 3, 4, 5], 'COL': [192, 192, 192, 192, 192], 'ROW': [328, 326, 324, 322, 320], 'NAME': ['GE_BrightCorner', 'DarkCorner', 'DarkCorner', 'A_21_P0014386', 'A_33_P3396872'], 'SPOT_ID': ['GE_BrightCorner', 'DarkCorner', 'DarkCorner', 'A_21_P0014386', 'A_33_P3396872'], 'CONTROL_TYPE': ['pos', 'pos', 'pos', 'FALSE', 'FALSE'], 'REFSEQ': [nan, nan, nan, nan, 'NM_001105533'], 'GB_ACC': [nan, nan, nan, nan, 'NM_001105533'], 'LOCUSLINK_ID': [nan, nan, nan, nan, 79974.0], 'GENE_SYMBOL': [nan, nan, nan, nan, 'CPED1'], 'GENE_NAME': [nan, nan, nan, nan, 'cadherin-like and PC-esterase domain containing 1'], 'UNIGENE_ID': [nan, nan, nan, nan, 'Hs.189652'], 'ENSEMBL_ID': [nan, nan, nan, nan, nan], 'TIGR_ID': [nan, nan, nan, nan, nan], 'ACCESSION_STRING': [nan, nan, nan, nan, 'ref|NM_001105533|gb|AK025639|gb|BC030538|tc|THC2601673'], 'CHROMOSOMAL_LOCATION': [nan, nan, nan, 'unmapped', 'chr7:120901888-120901947'], 'CYTOBAND': [nan, nan, nan, nan, 'hs|7q31.31'], 'DESCR

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine which columns contain the identifiers and gene symbols
# From previous output, we can see:
# - 'ID' column contains numeric identifiers matching our gene expression data
# - 'GENE_SYMBOL' column contains the gene symbols we need

# Create a mapping dataframe with the identifier and gene symbol columns
gene_mapping = get_gene_mapping(
    annotation=gene_annotation,
    prob_col='ID',
    gene_col='GENE_SYMBOL'
)

# Preview the mapping
print("Gene mapping preview:")
print(preview_df(gene_mapping))

# 2. Apply the gene mapping to convert probe-level expression to gene-level expression
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Preview the gene expression data after mapping
print("\nGene expression data after mapping:")
print(f"Number of genes: {len(gene_data)}")
print(f"Number of samples: {len(gene_data.columns)}")
print("Sample of first few genes:")
print(gene_data.head(5).iloc[:, :3])  # Show first 5 genes, first 3 samples


Gene mapping preview:
{'ID': ['5', '6', '7', '8', '12'], 'Gene': ['CPED1', 'BCOR', 'CHAC2', 'IFI30', 'GPR146']}

Gene expression data after mapping:
Number of genes: 29222
Number of samples: 32
Sample of first few genes:
          GSM2059404  GSM2059405  GSM2059406
Gene                                        
A1BG       -0.026845    0.292602   -0.127231
A1BG-AS1    0.000000    0.000000    0.000000
A1CF       -0.003664    0.000000    0.000000
A1CF-2      0.000000    0.000000    0.000000
A1CF-3      0.086574    0.000000    0.000000


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Normalized gene data shape: {normalized_gene_data.shape}")
print("First few genes with their expression values after normalization:")
print(normalized_gene_data.head())

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Check if trait data is available before proceeding with clinical data extraction
if trait_row is None:
    print("Trait row is None. Cannot extract trait information from clinical data.")
    # Create an empty dataframe for clinical features
    clinical_features = pd.DataFrame()
    
    # Create an empty dataframe for linked data
    linked_data = pd.DataFrame()
    
    # Validate and save cohort info
    validate_and_save_cohort_info(
        is_final=True, 
        cohort=cohort, 
        info_path=json_path, 
        is_gene_available=True, 
        is_trait_available=False,  # Trait data is not available
        is_biased=True,  # Not applicable but required
        df=pd.DataFrame(),  # Empty dataframe
        note="Dataset contains gene expression data but lacks clear trait indicators for Duchenne Muscular Dystrophy status."
    )
    print("Data was determined to be unusable due to missing trait indicators and was not saved")
else:
    try:
        # Get the file paths for the matrix file to extract clinical data
        _, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
        
        # Get raw clinical data from the matrix file
        _, clinical_raw = get_background_and_clinical_data(matrix_file)
        
        # Verify clinical data structure
        print("Raw clinical data shape:", clinical_raw.shape)
        
        # Extract clinical features using the defined conversion functions
        clinical_features = geo_select_clinical_features(
            clinical_df=clinical_raw,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        print("Clinical features:")
        print(clinical_features)
        
        # Save clinical features to file
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        clinical_features.to_csv(out_clinical_data_file)
        print(f"Clinical features saved to {out_clinical_data_file}")
        
        # 3. Link clinical and genetic data
        linked_data = geo_link_clinical_genetic_data(clinical_features, normalized_gene_data)
        print(f"Linked data shape: {linked_data.shape}")
        print("Linked data preview (first 5 rows, first 5 columns):")
        print(linked_data.iloc[:5, :5])
        
        # 4. Handle missing values
        print("Missing values before handling:")
        print(f"  Trait ({trait}) missing: {linked_data[trait].isna().sum()} out of {len(linked_data)}")
        if 'Age' in linked_data.columns:
            print(f"  Age missing: {linked_data['Age'].isna().sum()} out of {len(linked_data)}")
        if 'Gender' in linked_data.columns:
            print(f"  Gender missing: {linked_data['Gender'].isna().sum()} out of {len(linked_data)}")
        
        gene_cols = [col for col in linked_data.columns if col not in [trait, 'Age', 'Gender']]
        print(f"  Genes with >20% missing: {sum(linked_data[gene_cols].isna().mean() > 0.2)}")
        print(f"  Samples with >5% missing genes: {sum(linked_data[gene_cols].isna().mean(axis=1) > 0.05)}")
        
        cleaned_data = handle_missing_values(linked_data, trait)
        print(f"Data shape after handling missing values: {cleaned_data.shape}")
        
        # 5. Evaluate bias in trait and demographic features
        is_trait_biased = False
        if len(cleaned_data) > 0:
            trait_biased, cleaned_data = judge_and_remove_biased_features(cleaned_data, trait)
            is_trait_biased = trait_biased
        else:
            print("No data remains after handling missing values.")
            is_trait_biased = True
        
        # 6. Final validation and save
        is_usable = validate_and_save_cohort_info(
            is_final=True, 
            cohort=cohort, 
            info_path=json_path, 
            is_gene_available=True, 
            is_trait_available=True, 
            is_biased=is_trait_biased, 
            df=cleaned_data,
            note="Dataset contains gene expression data comparing Duchenne muscular dystrophy vs healthy samples."
        )
        
        # 7. Save if usable
        if is_usable and len(cleaned_data) > 0:
            os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
            cleaned_data.to_csv(out_data_file)
            print(f"Linked data saved to {out_data_file}")
        else:
            print("Data was determined to be unusable or empty and was not saved")
            
    except Exception as e:
        print(f"Error processing data: {e}")
        # Handle the error case by still recording cohort info
        validate_and_save_cohort_info(
            is_final=True, 
            cohort=cohort, 
            info_path=json_path, 
            is_gene_available=True, 
            is_trait_available=False,  # Mark as not available due to processing issues
            is_biased=True, 
            df=pd.DataFrame(),  # Empty dataframe
            note=f"Error processing data: {str(e)}"
        )
        print("Data was determined to be unusable and was not saved")

Normalized gene data shape: (20778, 32)
First few genes with their expression values after normalization:
          GSM2059404  GSM2059405  GSM2059406  GSM2059407  GSM2059408  \
Gene                                                                   
A1BG       -0.026845    0.292602   -0.127231   -0.125141   -0.076007   
A1BG-AS1    0.000000    0.000000    0.000000    0.251289    0.082415   
A1CF       -0.003664    0.000000    0.000000    0.000000    0.000000   
A2M         0.103392    0.000000   -0.070657    0.483920   -0.159715   
A2M-AS1    -0.022907   -0.019728   -0.104097   -0.203670   -0.555398   

          GSM2059409  GSM2059410  GSM2059411  GSM2059412  GSM2059413  ...  \
Gene                                                                  ...   
A1BG       -0.009207   -0.082495    0.189666   -0.237983   -0.006939  ...   
A1BG-AS1    0.065617    0.000000    0.000000   -0.050387   -0.103464  ...   
A1CF        0.000000   -0.243397    0.000000    0.000000    0.000000  ...   
A2M 

Normalized gene data saved to ../../output/preprocess/Esophageal_Cancer/gene_data/GSE77790.csv
Raw clinical data shape: (3, 33)
Clinical features:
                   GSM2059404  GSM2059405  GSM2059406  GSM2059407  GSM2059408  \
Esophageal_Cancer         0.0         0.0         0.0         0.0         0.0   

                   GSM2059409  GSM2059410  GSM2059411  GSM2059412  GSM2059413  \
Esophageal_Cancer         0.0         0.0         0.0         0.0         0.0   

                   ...  GSM2059426  GSM2059427  GSM2059428  GSM2059429  \
Esophageal_Cancer  ...         0.0         0.0         0.0         0.0   

                   GSM2059430  GSM2059431  GSM2059432  GSM2059433  GSM2059434  \
Esophageal_Cancer         1.0         1.0         0.0         0.0         0.0   

                   GSM2059435  
Esophageal_Cancer         0.0  

[1 rows x 32 columns]
Clinical features saved to ../../output/preprocess/Esophageal_Cancer/clinical_data/GSE77790.csv
Linked data shape: (32, 20779)
L

  Samples with >5% missing genes: 0


Data shape after handling missing values: (32, 20779)
For the feature 'Esophageal_Cancer', the least common label is '1.0' with 2 occurrences. This represents 6.25% of the dataset.
The distribution of the feature 'Esophageal_Cancer' in this dataset is severely biased.

Data was determined to be unusable or empty and was not saved
